In [ ]:
from sklearn.datasets import fetch_20newsgroups
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import SGDClassifier
import numpy as np

In [ ]:
remove=('footers', 'quotes')
categories = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space',
    ]
dataset = fetch_20newsgroups(subset='train', categories=categories, random_state=22, remove=remove)
testset = fetch_20newsgroups(subset='test', categories=categories, random_state=22, remove=remove)

In [ ]:
# 简单清洗
def clean(data):
    pun = """,.?!;:<>\"\'()[]"""
    stop = """-_="""
    data = [z.lower().replace('\n', ' ') for z in data]
    for p in pun:
        data = [z.replace(p, ' ') for z in data]
    for s in stop:
        data = [z.replace(s, ' ') for z in data]
    data = [z.split() for z in data]
    return data
data_train = clean(dataset.data)
data_test = clean(testset.data)
# 读取额外语料
with open('text8', 'r') as f:
    ex_text = f.readlines()
ex_data = [z.split() for z in ex_text]

In [ ]:
corpus.extend(data_train)
corpus.extend(ex_data)

In [ ]:
model = Word2Vec(corpus,
                 min_count=10,
                 iter=3, 
                 size=256, 
                 window=5,
                 workers=8)

In [ ]:
def ave_w2v(content):
    if not isinstance(content, np.ndarray):
        content = np.array(content)
    word_cnt = 0
    d2v = np.zeros(256)
    for word in content:
        if word not in model:
            continue
        d2v = np.add(model[word], d2v)
        word_cnt += 1
    return np.divide(d2v, word_cnt)

data_train_v = map(ave_w2v, data_train)
data_test_v = map(ave_w2v, data_test)

In [ ]:
sgd_clf = SGDClassifier(loss='log', penalty=None, )
sgd_clf.fit(data_train_v, dataset.target)

In [ ]:
sgd_clf.score(data_test_v, testset.target)